In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import requests

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set


In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [4]:
# original propmt
system_message = """
You are an assistant that is very good at determining what tool to use to solve queries.
"""

ai_programming = """

TOOLS

You have two tools:

1. A calculator tool that can perform basic arithmetic operations, such as addition, subtraction, multiplication, and division. If this tool is used then respond as detailed in the OUTPUT section with "next": "do_calculation"
2. A jokes tool that can provide a light-hearted joke for the audience reqested. If this tool is used then respond as detailed in <OUTPUT> with "next": "do_joke" and "audience": "audience requested"


OUTPUT

For a user query, determine the best tool. If none of the tools are appropriate, respond with "None".

EXAMPLES

    First Example
    
    <OUTPUT>
    {"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 5, "num2": 5, "operation": "addition"}} 
    <OUTPUT>

    Second Example

    <OUTPUT>
    {"tool": "joke", "next": "do_joke", "audience": "Pythonistas" }} 
    <OUTPUT>
    
Thank you for your help.
"""

system_message += ai_programming

In [5]:
user_prompt = "What is 5 times 9?"
# user_prompt = "Tell me a joke when I am doing stand up at a Builders Conference"

# user_prompt = "Tell me a joke when I am doing stand up at a Builders Conference"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

In [6]:
class Agent:
    def __init__(self, model="gpt-3.5-turbo", prompts=prompts):
        self.model = model
        self.prompt = prompts

    def get_tool(self):
        response = openai.chat.completions.create(
            model=self.model, messages=self.prompt
        )
        output = response.choices[0].message.content.replace("\n", "")
        json_output = output.replace("<OUTPUT>", "").strip()
        json_output = json.loads(json_output)
        # print(json_output)
        return json_output

    def do_calculation(self, num1, num2, operation):
        if operation == "addition":
            return num1 + num2
        elif operation == "subtraction":
            return num1 - num2
        elif operation == "multiplication":
            return num1 * num2
        elif operation == "division":
            return num1 / num2

    def do_joke(self, audience):
        return f"Joke for {audience}"

    def give_answer(self):
        output = self.get_tool()
        if output["next"] == "do_calculation":
            num1 = output["arguments"]["num1"]
            num2 = output["arguments"]["num2"]
            operation = output["arguments"]["operation"]
            answer = self.do_calculation(num1, num2, operation)
            return answer
        elif output["next"] == "do_joke":
            audience = output["audience"]
            answer = self.do_joke(audience)
            return answer

In [7]:
agent = Agent()
output = agent.get_tool()
print(output)

{'tool': 'calculator', 'next': 'do_calculation', 'arguments': {'num1': 5, 'num2': 9, 'operation': 'multiplication'}}


In [8]:
answer = agent.give_answer()
print(answer)

45
